# LLaMA Factory Colab Tutorial

Please use a **free** Tesla T4 Colab GPU to run this!

Project homepage: https://github.com/hiyouga/LLaMA-Factory

### **这是我llama board项目的官方colab教程，复制过来的副本。--mrking**

## Install Dependencies

In [ ]:
%rm -rf LLaMA-Factory
!git clone https://github.com/tking007/LLaMA-Factory.git
%cd LLaMA-Factory
%ls
!pip install .

### Check GPU environment

In [ ]:
import torch
try:
  assert torch.cuda.is_available() is True
except AssertionError:
  print("Please set up a GPU before using LLaMA Factory: https://medium.com/mlearning-ai/training-yolov4-on-google-colab-316f8fff99c6")

Please set up a GPU before using LLaMA Factory: https://medium.com/mlearning-ai/training-yolov4-on-google-colab-316f8fff99c6


### Log in with Hugging Face account to upload model (Optional)

In [ ]:
!huggingface-cli login

## Fine-tune model via LLaMA Board

In [ ]:
# !CUDA_VISIBLE_DEVICES=0 python src/train_web.py

In [ ]:
from llmtuner import create_ui

create_ui().queue().launch(share=True, debug=True)

AttributeError: module 'transformers.utils' has no attribute 'logging'

In [ ]:
!pip install modelscope

In [ ]:
from modelscope.hub.api import HubApi

YOUR_ACCESS_TOKEN = '297bc887-49b3-4335-b95b-23ec301bad8a'
# 请注意ModelScope平台针对SDK访问和git访问两种模式，提供两种不同的访问令牌(token)。此处请使用SDK访问令牌。


api = HubApi()
api.login(YOUR_ACCESS_TOKEN)
api.push_model(
    model_id="/content/LLaMA-Factory/mrking_Qwen_1.5_7B",
    model_dir="/content/LLaMA-Factory/mrking_Qwen_1.5_7B" # 本地模型目录，要求目录中必须包含configuration.json
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Fine-tune model via Command Line

In [ ]:
from llmtuner import run_exp
run_exp(dict(
  stage="sft",
  do_train=True,
  model_name_or_path="Qwen/Qwen1.5-0.5B-Chat",
  dataset="identity,alpaca_gpt4_en,alpaca_gpt4_zh",
  template="qwen",
  finetuning_type="lora",
  lora_target="all",
  output_dir="test_identity",
  per_device_train_batch_size=4,
  gradient_accumulation_steps=4,
  lr_scheduler_type="cosine",
  logging_steps=10,
  save_steps=100,
  learning_rate=1e-4,
  num_train_epochs=5.0,
  max_samples=500,
  max_grad_norm=1.0,
  fp16=True,
))

In [ ]:
import os
print(os.getcwd())
os.chdir("/content/drive/MyDrive/LLaMA-Factory")
print(os.getcwd())

/content/LLaMA-Factory
/content/drive/MyDrive/LLaMA-Factory


In [ ]:
import os
print(os.getcwd())
os.chdir("/content/LLaMA-Factory")
print(os.getcwd())

/content
/content/LLaMA-Factory
/content/LLaMA-Factory
/content/LLaMA-Factory


In [ ]:
!cd /content/drive/MyDrive/LLaMA-Factory

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
chdir: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


In [ ]:
import os
print(os.getcwd())
os.chdir("/content/drive/MyDrive/LLaMA-Factory")

/content


### Infer the fine-tuned model

In [ ]:
from llmtuner import ChatModel
chat_model = ChatModel(dict(
  model_name_or_path="Qwen/Qwen1.5-0.5B-Chat",
  adapter_name_or_path="test_identity",
  finetuning_type="lora",
  template="qwen",
))
messages = []
while True:
  query = input("\nUser: ")
  if query.strip() == "exit":
    break
  if query.strip() == "clear":
    messages = []
    continue

  messages.append({"role": "user", "content": query})
  print("Assistant: ", end="", flush=True)
  response = ""
  for new_text in chat_model.stream_chat(messages):
    print(new_text, end="", flush=True)
    response += new_text
  print()
  messages.append({"role": "assistant", "content": response})

### Merge LoRA weights

In [ ]:
from llmtuner import export_model
export_model(dict(
  model_name_or_path="Qwen/Qwen1.5-7B",
  adapter_name_or_path="test_identity",
  finetuning_type="lora",
  template="qwen",
  export_dir="test_exported",
  export_hub_model_id="jtjt520j/Qwen_1.5_7b_text_to_sql_mrking",
))

[INFO|tokenization_utils_base.py:2046] 2024-03-07 14:53:43,007 >> loading file vocab.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen1.5-7B/snapshots/e52fa2ef47411cc8bc9f752d1d8d9072b37742e7/vocab.json
[INFO|tokenization_utils_base.py:2046] 2024-03-07 14:53:43,008 >> loading file merges.txt from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen1.5-7B/snapshots/e52fa2ef47411cc8bc9f752d1d8d9072b37742e7/merges.txt
[INFO|tokenization_utils_base.py:2046] 2024-03-07 14:53:43,009 >> loading file added_tokens.json from cache at None
[INFO|tokenization_utils_base.py:2046] 2024-03-07 14:53:43,010 >> loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen1.5-7B/snapshots/e52fa2ef47411cc8bc9f752d1d8d9072b37742e7/special_tokens_map.json
[INFO|tokenization_utils_base.py:2046] 2024-03-07 14:53:43,011 >> loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen1.5-7B/snapshots/e52fa2ef47411cc8bc9

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

[INFO|modeling_utils.py:3992] 2024-03-07 14:53:45,632 >> All model checkpoint weights were used when initializing Qwen2ForCausalLM.

[INFO|modeling_utils.py:4000] 2024-03-07 14:53:45,633 >> All the weights of Qwen2ForCausalLM were initialized from the model checkpoint at Qwen/Qwen1.5-7B.
If your task is similar to the task the model of the checkpoint was trained on, you can already use Qwen2ForCausalLM for predictions without further training.
[INFO|configuration_utils.py:800] 2024-03-07 14:53:45,919 >> loading configuration file generation_config.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen1.5-7B/snapshots/e52fa2ef47411cc8bc9f752d1d8d9072b37742e7/generation_config.json
[INFO|configuration_utils.py:845] 2024-03-07 14:53:45,921 >> Generate config GenerationConfig {
  "bos_token_id": 151643,
  "eos_token_id": 151643,
  "max_new_tokens": 2048
}



03/07/2024 14:53:45 - INFO - llmtuner.model.adapter - Fine-tuning method: LoRA


INFO:llmtuner.model.adapter:Fine-tuning method: LoRA


ValueError: Can't find 'adapter_config.json' at 'test_identity'